In [ ]:
#!pip install sentence_transformers
#!pip install ag2[retrievechat]
#!pip install pyautogen==0.7.2
#https://microsoft.github.io/autogen/0.2/docs/notebooks/agentchat_RetrieveChat/

# Setting base for Retrive Chat

In [8]:
import json
import os

import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS


In [9]:
print("Accepted file formats for `docs_path`:")
print(TEXT_FORMATS)

Accepted file formats for `docs_path`:
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf', 'mdx']


In [10]:
api_key = os.environ["GEMINI_API_KEY"]
config_list = [
    {
        "model": "gemini-1.5-pro-latest",
        "api_key": api_key,
        "api_type": "google"
    }
]

In [ ]:
#import agentops
#agentops.init("putYourKey")

In [11]:
# 1. create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)


In [12]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Extract text from local PDF files
pdf_files = ["ComprehensiveWarranty.pdf"]
#docs_content = [extract_text_from_pdf(file) for file in pdf_files]

from autogen.retrieve_utils import TEXT_FORMATS,extract_text_from_pdf
docs_content= extract_text_from_pdf("ComprehensiveWarranty.pdf")

In [19]:
# Define the RetrieveUserProxyAgent instance
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": ["autogen-docs/ComprehensiveWarranty.pdf"],
        "docs_content": docs_content,  # Provide the extracted content directly
        "chunk_token_size": 200,
        "model": "gemini-1.5-pro-latest",
        "vector_db": "chroma",
        "overwrite": True,
        "get_or_create": True,
    },
    code_execution_config=False,
)

# Process Starts for Scenario 1 : Generate code based off docstrings w/o human feedback

In [20]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Write a python code to check if warantee is covered or not. input will be year of purchase"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)
#agentops.end_session("Success")

Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	  ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2025-03-22 16:22:31,244 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 6 chunks.
2025-03-22 16:22:31,278 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6
Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['735b1899', '2add3c40', '4365ed84', '9d11a270', '5aaaa884', 'dcd986b8']]
Adding content of doc 735b1899 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc 2add3c40 to context.


...

Adding content of doc 5aaaa884 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc dcd986b8 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Write a python code to check if warantee is covered or not. input will be year of purchase

Context is: mandatory for anti-leak split AC series. 
• The special warranty shall not be in any case extended towards any monetary 
compensation whatsoever or return or replacement of the product  
• In the rare event where part needed for the air conditioner is not available or if the 
machine can’t be repaired, c

Number of requested results 60 is greater than number of elements in index 6, updating n_results = 6


VectorDB returns doc_ids:  [['735b1899', '2add3c40', '4365ed84', '9d11a270', '5aaaa884', 'dcd986b8']]
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (975495cf-d231-45a8-8d46-a52722c48aec): Termination message condition on agent 'assistant' met


In [ ]:
#agentops.end_session("Success")

# Process Starts for Scenario 2 : Answer a question based off docstrings w/o human feedback

In [22]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()
ragproxyagent.task="qa"
qa_problem = "Is 5 year warantee applicable if I purchased the AC on January 2023"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6
Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['4365ed84', '735b1899', '2add3c40', '9d11a270', '5aaaa884', 'dcd986b8']]
Adding content of doc 4365ed84 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc 735b1899 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


...

Adding content of doc 9d11a270 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc 5aaaa884 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc dcd986b8 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Is 5 year warantee applicable if I purchased the AC on January 2023

Context is:  
Internal 
 
5 YEAR COMPREHENSIVE AIR CONDITIONER WARRANTY OFFER 
mandatory for anti-leak split AC series. 
• The special warranty shall not be in any case extended towards any monetary 
compensation whatsoever or return or replacement of the product  
• In the rare event where part needed for the air conditioner is not avail

# Process Starts for Scenario 3 : Generate code based off docstrings w/ human feedback

In [23]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
ragproxyagent.task="code"
code_problem = "Write a python code to check if warantee is covered or not. input will be year of purchase"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=code_problem)

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6
Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['735b1899', '2add3c40', '4365ed84', '9d11a270', '5aaaa884', 'dcd986b8']]
Adding content of doc 735b1899 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc 2add3c40 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


...

ragproxyagent (to assistant):

What is the ouput if my purchased date is January 1st 2025?

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

The output of the code if the purchase date is January 1st, 2025 will be:

```
Warranty is covered.
```

The code currently only checks the year. Since 2025 is after the start date of the warranty (September 1st, 2024), the function returns `True`.


--------------------------------------------------------------------------------


Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Could you please modify the code with exact day and month along with year?


ragproxyagent (to assistant):

Could you please modify the code with exact day and month along with year?

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

```python
from datetime import datetime

def is_warranty_covered(purchase_date_str):
    """
    Checks if the warranty is covered for a Godrej AC based on the purchase date.

    Args:
        purchase_date_str: The purchase date as a string in "YYYY-MM-DD" format.

    Returns:
        True if the warranty is covered, False otherwise.
    """
    try:
        purchase_date = datetime.strptime(purchase_date_str, "%Y-%m-%d").date()
    except ValueError:
        return False  # Handle invalid date format

    warranty_start_date = datetime(2024, 9, 1).date()  # Warranty starts from Sept 1st, 2024
    #No information on when warranty ends, so assuming current date.
    current_date = datetime.now().date()

    if warranty_start_date <= purchase_date <= current_date:
     

Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (cd6108bf-c17a-4f24-8ed1-65f5fb14e3a8): User requested to end the conversation


# Process Starts for Scenario 4: Answer a question based off docstrings w/ human feedback

In [24]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
ragproxyagent.task="qa"
qa_problem = "Is 5 year warantee applicable if I purchased the AC on January 2023"
chat_result = ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=qa_problem
)  # type "exit" to exit the conversation

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6
Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['4365ed84', '735b1899', '2add3c40', '9d11a270', '5aaaa884', 'dcd986b8']]
Adding content of doc 4365ed84 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


Adding content of doc 735b1899 to context.


Model gemini-1.5-pro-latest not found. Using cl100k_base encoding.


...

Adding content of doc dcd986b8 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Is 5 year warantee applicable if I purchased the AC on January 2023

Context is:  
Internal 
 
5 YEAR COMPREHENSIVE AIR CONDITIONER WARRANTY OFFER 
mandatory for anti-leak split AC series. 
• The special warranty shall not be in any case extended towards any monetary 
compensation whatsoever or return or replacement of the product  
• In the rare event where part needed for the air conditioner is not avail

Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  I purchased AC on 1st January 2025. Can I claim the warantee? Upto whjich date warantee valid?


ragproxyagent (to assistant):

I purchased AC on 1st January 2025. Can I claim the warantee? Upto whjich date warantee valid?

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

Yes, if you purchased the AC on January 1, 2025, and it was an anti-leak split AC series purchased through an authorized dealer and preferably installed by Godrej or an authorized service provider, then the 5-year warranty applies.  It would be valid until January 1, 2030 (5 years from the purchase date).


--------------------------------------------------------------------------------


Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (d5a71593-f861-448c-b91b-9382d7fc70f6): User requested to end the conversation
